In [52]:
import numpy as np
import stan
import arviz as az

# stupid stan problems
import nest_asyncio
nest_asyncio.apply()

# true param
alpha_true = 2.3,
beta_true = 4.0,
sigma_true = 2.0,
N = 100

# simulation
np.random.seed(42)
x = np.random.normal(size=N)
y = alpha_true + beta_true * x + sigma_true * np.random.normal(size=N)

In [53]:
stanCode = """
data {
  int<lower=0> N;
  vector[N] x;
  vector[N] y;
}
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma_sq;
}
transformed parameters {
  real<lower=0> sigma = sqrt(sigma_sq);
}
model {
  sigma_sq ~ inv_gamma(1, 1);  // prior on variance
  alpha ~ normal(0, 10);
  beta ~ normal(0, 10);
  y ~ normal(alpha + beta * x, sigma);  // likelihood
}
"""

In [54]:
# Define data first
data = {"N": N, "x": x, "y": y}

# Build the model with data
model = stan.build(stanCode, data=data)

# Sample
fit = model.sample(num_chains=4, num_samples=2000)

az.summary(az.from_pystan(fit))

Building...



Building: found in cache, done.Sampling:   0%
Sampling:  25% (3000/12000)
Sampling:  50% (6000/12000)
Sampling:  75% (9000/12000)
Sampling: 100% (12000/12000)
Sampling: 100% (12000/12000), done.
Messages received during sampling:
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.27 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan__2qigylb/model_74j73ceb.stan', line 19, column 2 to column 38)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often 

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,2.317,0.192,1.959,2.683,0.002,0.002,6909.0,5804.0,1.0
beta,3.713,0.208,3.327,4.117,0.002,0.002,7805.0,5904.0,1.0
sigma_sq,3.615,0.511,2.716,4.584,0.006,0.006,7166.0,5819.0,1.0
sigma,1.897,0.133,1.648,2.141,0.002,0.001,7166.0,5819.0,1.0


### Step 4: Analyze Results for N=100

Posterior summaries should be close to the true values:

- **α**: approximately 2.3
- **β**: approximately 4.0
- **σ**: approximately 2.0

Also compute the 95% credible intervals.

### Step 5: Repeat with N=1000

Increase the sample size and rerun the simulation and model fitting.

In [55]:
N_large = 1000;
x_large = np.random.normal(size=N_large);
y_large = alpha_true + beta_true * x_large + sigma_true * np.random.normal(size=N_large);

# create new data dictionary
data_large = {"N": N_large, "x": x_large, "y": y_large};
model_large = stan.build(stanCode, data=data_large)

# fit the model again
fit_large = model_large.sample(num_chains=4, num_samples=2000);

# check diagnostics for larger data
az.summary(az.from_pystan(fit_large))


Building...



Building: found in cache, done.Sampling:   0%
Sampling:  25% (3000/12000)
Sampling:  50% (6000/12000)
Sampling:  75% (9000/12000)
Sampling: 100% (12000/12000)
Sampling: 100% (12000/12000), done.
Messages received during sampling:
  Gradient evaluation took 0.000146 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.46 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.000126 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.26 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan__2qigylb/model_74j73ceb.stan', line 19, column 2 to column 38)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs ofte

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,2.366,0.062,2.253,2.484,0.001,0.001,7563.0,5508.0,1.0
beta,3.929,0.063,3.814,4.048,0.001,0.001,8352.0,5934.0,1.0
sigma_sq,3.895,0.174,3.588,4.236,0.002,0.002,8354.0,6044.0,1.0
sigma,1.973,0.044,1.894,2.058,0.000,0.000,8354.0,6044.0,1.0
